In [2]:
import dash

from dash import dcc, html
from dash.dependencies import Input, Output
import plotly.express as px
import pandas as pd
import matplotlib.pyplot as plt
import psycopg2

from dotenv import load_dotenv
import os

In [3]:
load_dotenv()

host = os.getenv('HOST')
port = os.getenv('PORT')
user = os.getenv('USER')
password = os.getenv('PASSWORD')
database= os.getenv('DATABASE')

In [5]:
connection = psycopg2.connect(host=host, port=port, database=database, user=user, password=password)
print("Verbindung zur Datenbank erfolgreich hergestellt.")

# Einen Cursor erstellen, um SQL-Abfragen auszuführen
cursor = connection.cursor()

# SQL SELECT-Abfrage für die Stationen
select_value_query = """
    SELECT v.*, b.ort
    FROM value v
    JOIN box_id b ON v.senseid_fk = b.senseid;
"""
cursor.execute(select_value_query)
values = cursor.fetchall()

df = pd.DataFrame(values, columns=[desc[0] for desc in cursor.description])

Verbindung zur Datenbank erfolgreich hergestellt.


In [20]:
stations = df['ort'].unique()
stations_options = [{'label': 'Alle Stationen', 'value': 'all'}] + [{'label': station, 'value': station} for station in stations]

In [12]:
#Nur für tests benutzten, falls Datenbank nicht verfügbar ist
df = pd.read_csv("valueDaten.csv")
df['time'] = pd.to_datetime(df['time'],format='mixed')

In [7]:
df['date'] = df['time'].dt.date

In [16]:
stations

array(['Dortmund', 'Mersch', 'Ettlingen', 'Hamburg', 'Stuttgart',
       'München', 'Berlin', 'Jena'], dtype=object)

In [9]:
df_agg = df[["index","date","temperature","luftfeuchtigkeit","pm2_5","pm10"]]

In [10]:
daily_avg_df = df_agg.groupby('date').mean().reset_index()

In [13]:
daily_avg_df.head()

,date,index,temperature,luftfeuchtigkeit,pm2_5,pm10
0,2022-04-07,1.216997e+06,13.125532,77.317901,0.786357,2.414228
1,2022-04-08,1.216547e+06,8.524660,83.127407,1.811914,4.149907
2,2022-04-09,1.216396e+06,8.136848,80.464242,3.160295,5.832356
3,2022-04-10,1.215659e+06,8.463058,75.954083,2.720927,4.529338
4,2022-04-11,1.215233e+06,11.867487,63.463709,2.251815,3.838562


In [24]:
# Dash-App initialisieren
app = dash.Dash(__name__)

# Layout der Dash-App definieren
app.layout = html.Div(children=[
    html.H1(
        children='Umweltüberwachung Dashboard',
        style={'background-color': 'white'}
    ),

    html.Div(
        children='Wählen Sie eine Station und eine Kennzahl aus, um die Daten anzuzeigen.',
        style={'background-color': 'white'}
    ),

    dcc.Dropdown(
        id='station-dropdown',
        options=stations_options,
        value='all'
    ),

    html.Div(
        children='Wählen Sie eine Kennzahl:',
        style={'background-color': 'white', 'marginTop': 20}
    ),
    
    dcc.RadioItems(
        id='metric-radio',
        options=[
            {'label': 'Durchschnitt', 'value': 'mean'},
            {'label': 'Median', 'value': 'median'},
            {'label': 'Varianz', 'value': 'var'},
            {'label': 'Standardabweichung', 'value': 'std'}
        ],
        value='mean',
        labelStyle={'display': 'inline-block', 'marginRight': 10,"background-color" : "white"}
    ),

    dcc.Graph(id='daily-temperature-graph'),
    dcc.Graph(id='daily-luftfeuchtigkeit-graph'),
    dcc.Graph(id='daily-pm-graph')
])

# Callback zur Aktualisierung der Diagramme
@app.callback(
    [Output('daily-temperature-graph', 'figure'),
     Output('daily-luftfeuchtigkeit-graph', 'figure'),
     Output('daily-pm-graph', 'figure')],
    [Input('station-dropdown', 'value'),
     Input('metric-radio', 'value')]
)
def update_graphs(selected_station, selected_metric):
    # Filtere den DataFrame nach der ausgewählten Station
    if selected_station != 'all':
        filtered_df = df[df['station_name'] == selected_station]
    else:
        filtered_df = df.copy()

    # Erstelle eine neue Spalte für das Datum (ohne Uhrzeit)
    filtered_df['date'] = filtered_df['time'].dt.date

    # Bereinige den DataFrame von NaN-Werten
    filtered_df = filtered_df.dropna(subset=['temperature', 'luftfeuchtigkeit', 'pm2_5', 'pm10'])

    # Berechne die täglichen Kennzahlen
    daily_df = filtered_df.groupby('date').agg({
        'temperature': selected_metric,
        'luftfeuchtigkeit': selected_metric,
        'pm2_5': selected_metric,
        'pm10': selected_metric
    }).reset_index()

    # Erstelle Diagramme
    temp_fig = px.line(daily_df, x='date', y='temperature', title=f'Tägliche {selected_metric.capitalize()} Temperatur')
    humidity_fig = px.line(daily_df, x='date', y='luftfeuchtigkeit', title=f'Tägliche {selected_metric.capitalize()} Luftfeuchtigkeit')
    pm_fig = px.line(daily_df, x='date', y=['pm2_5', 'pm10'], title=f'Tägliche {selected_metric.capitalize()} PM2.5 und PM10')

    return temp_fig, humidity_fig, pm_fig

# App ausführen
if __name__ == '__main__':
    app.run_server(debug=True, host='127.0.0.1', port=8050)

---------------------------------------------------------------------------
KeyError                                  Traceback (most recent call last)
File c:\Users\Benjamin\anaconda3\Lib\site-packages\pandas\core\indexes\base.py:3791, in Index.get_loc(
    self=Index(['index', 'time', 'temperature', 'luftfeuc...enseid_fk', 'ort', 'date'],
      dtype='object'),
    key='station_name'
)
   3790 try:
-> 3791     return self._engine.get_loc(casted_key)
        casted_key = 'station_name'
        self = Index(['index', 'time', 'temperature', 'luftfeuchtigkeit', 'pm10', 'pm2_5',
       'senseid_fk', 'ort', 'date'],
      dtype='object')
   3792 except KeyError as err:

File index.pyx:152, in pandas._libs.index.IndexEngine.get_loc()

File index.pyx:181, in pandas._libs.index.IndexEngine.get_loc()

File pandas\_libs\hashtable_class_helper.pxi:7080, in pandas._libs.hashtable.PyObjectHashTable.get_item()

File pandas\_libs\hashtable_class_helper.pxi:7088, in pandas._libs.hashtable.PyObjectHas